# Merger Components

The first part of the data cleaning tool is the merger. It is intended to help discover and merge data from multiple 
sources. For data sources, we mean table-based data, primarily sitting in Excel spreadsheets, but tool is designed so that it should be able to deal with other table-based formats, such as CVS.

We will be loading sources from our parent directory, so we need to tweak the package path.

In [ ]:
from dataclasses import asdict
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Defining Tables

The Front Door Data Collaboration merge tool uses a concept of Data Sources with defined Columns. Data Sources and Columns have fixed names, and a few additional parameters. They can both have a set of matching rules so that any scanned sources can be matched with a defined Data Source. For example, say an scanned Excel workbook has a worksheet named "list_1" - this could be matched with a Data Source called "List 1" based on the configuration. 

Default matchers are defined when loading the configuration file, but additional options can be added in the configuration. 

Let's start by loading the configuration for Annex A.


In [ ]:
from fddc.annex_a.merger import configuration

data_sources = configuration.parse_datasources("../config/annex-a-merge.yml")
asdict(data_sources[0])

## Discovering data

The first part of the process is file discovery. This is a very simple glob based pattern to help discover target files.


In [ ]:
from fddc.annex_a.merger import file_scanner

files = file_scanner.find_input_files("../examples/*.xls*")
files

## Extracting a sort key

Often we want to make sure files are sorted in the right order based on, quite commonly, the year. We can see that examples above sort lexically in the 'wrong' order.

In [ ]:
files = file_scanner.find_input_files(
    file_scanner.ScanSource("../examples/*.xls*", sort_keys=[r'/.*?-(?P<cat>\w)-(?P<year>\d+).*/\g<year>-\g<cat>/i'])
)
files

And yeah... I'm showing off with regular expressions, but just wanted to demonstrate that we can used named groups for quite advanced syntax.

Having discovered our files, we can then scan these to see if we can find any data for loading.

In [ ]:
from fddc.annex_a.merger import workbook_util

worksheets = workbook_util.find_worksheets(files[0])
display(f"We found {len(worksheets)} worksheets.")

asdict(worksheets[5])

# Matching data to a source

Once we have discovered our data, we can try to match it to our source definitions. This first matches the worksheets to a Data Source item based on the worksheet name.

In [ ]:
from fddc.annex_a.merger import matcher

matched_sheets, unmatched_sheets = matcher.match_data_sources(worksheets, data_sources)

display(f"We found {len(matched_sheets)} matched sheets.")

asdict(matched_sheets[0])

A 'MatchedSheet' is just a container with a worksheet item and a datasource item. 

The `match_data_sources` function also returns a list of sheets that were discovered but if not matched:
    

In [ ]:
display(f"We found {len(unmatched_sheets)} unmatched sheets.")

asdict(unmatched_sheets[0])


## Match columns

The next step is to try to match the columns in the two datasets, again using the names.

In [ ]:
sheet_with_headers = matcher.match_columns(matched_sheets)
asdict(sheet_with_headers[0])

## Reviewing the matches

We now have a matched dataset - however, the automatic matching may not have been perfect, we can write a report showing what was matched and what was not:

In [ ]:
from fddc.annex_a.merger import matcher_report

matcher_report.column_report(
    sheet_with_headers, 
    unmatched_sheets, 
    "my-match-report.xlsx")


## Improving the matches

How about that? It actually returns a Pandas Dataframe.

The Excel report can be edited to fix any matches or add files that may have been missed. If you manually add files or delete all the `column_name` values for an entire table, then that table will be rescanned.

Let's try a few, just to see what happens.


In [ ]:
import pandas as pd
rows = [
    {
        # This will be completely scanned for matches
        "filename": "../examples/example-A-2005.xls",
        "sort_key": "2005-A",
    },
    {
        # This table should be auto-matched
        "filename": "../examples/example-B-2004.xlsx",
        "sort_key": "2004-B",
        "sheetname": "List_1",
        "table": "List 1"
    },
    {
        # The next records will manually map a few columns - I'm purposfully going to change the mappings
        # so we can see that it is being applied
        "filename": "../examples/example-B-2004.xlsx",
        "sort_key": "2004-B",
        "sheetname": "List_2",
        "table": "List 2",
        "column_name": "Child Unique ID",
        "header_name": "Child Unique ID"
    },
    {
        "filename": "../examples/example-B-2004.xlsx",
        "sort_key": "2004-B",
        "sheetname": "List_2",
        "table": "List 2",
        "column_name": "Gender",
        "header_name": "Ethnicity"
    },
    {
        "filename": "../examples/example-B-2004.xlsx",
        "sort_key": "2004-B",
        "sheetname": "List_2",
        "table": "List 2",
        "column_name": "Ethnicity",
        "header_name": "Gender"
    }
]
report = pd.DataFrame(rows)
report

In [ ]:
sheet_with_headers, unmatched_sheets = matcher_report.process_report(report, data_sources=data_sources)
display(f"We configured {len(sheet_with_headers)} sheets")
df = matcher_report.column_report(
    sheet_with_headers, 
    unmatched_sheets
)
df[["filename","sort_key", "sheetname","table"]].drop_duplicates()

We can also see that in the second file that we manually matched, has been included. List 1 has been automatically matched since we did not include any values ourselves. However, List 2 has only matched the columns we provided, including the wrong matches. The unmatched values are included in the report so they can be matched later if required.

In [ ]:
df[df.filename=="../examples/example-B-2004.xlsx"][["table","column_name","header_name"]].drop_duplicates()

## Merging data

Now that we have our merge configuration, we're ready to merge the contents of the different tables. 

The merge process is relatively simple. The data is read from each source, then normalised so that all the different sources are comparable. The data is sorted by the 'sort_key' from the configuration, and then deduplicated based on the unique columns in the configuration. When multiple records are found, the latest record is kept based on the sort_key order.

Finally, as we need columns to be comparable to deduplicate, we can also force some columns to conform to be a date, for example. This is also set in the configuration. 

Let's manually merge our List 1.

In [ ]:
# Filter List 2 from our previous results
list_2 = [s for s in sheet_with_headers if s.sheet.source_config.name == "List 2"]
data_source_config = next(iter([ds for ds in data_sources if ds.name == "List 2"]))

display(len(list_2))
data_source_config

In [ ]:
from fddc.datatables import load, normalise, merge


df_1 = load.load_dataframe(list_2[0].sheet.sheet_detail)
df_1

In [ ]:
df_2 = load.load_dataframe(list_2[1].sheet.sheet_detail)
df_2

## We now need to make sure that both dataframes have the same columns 

Ok, so that's not very exciting for our example - but since we're remapping some columns we'll get an idea of how it works.

One of the advantages of our configuration objects is that they generate a column name map between what is in the source data and what we expect.

In [ ]:
print("The first one isn't very exciting because it automatically matched all of the columns\n")
display(list_2[0].column_map())

print("\n\nThe second shows clearly how we remapped some of the columns\n")
display(list_2[1].column_map())


In [ ]:
df_1_normalised = normalise.normalise_dataframe(df_1, data_source_config.column_names(), list_2[0].column_map())
df_1_normalised

In [ ]:
df_2_normalised = normalise.normalise_dataframe(df_2, data_source_config.column_names(), list_2[1].column_map())
df_2_normalised

Note how gender and ethnicity is swapped, and all the other columns are blank as we did not map those. There is an option to retain those that already have the correct names.

In [ ]:
df_2_normalised = normalise.normalise_dataframe(df_2, data_source_config.column_names(), list_2[1].column_map(), 
                                     only_retain_mapped=False)
df_2_normalised

## Clean columns

We can see that somehow the date for 1008 has been corrupted and it's upset the entire column - it would also stop us from merging these if we had a unique flag on that column as we couldn't accurately compare the values. We therefore use the settings from our configuration to make sure both tables have the correct types.


In [ ]:
df_1_cleaned = normalise.clean_datatypes(df_1_normalised, data_source_config.columns)
df_2_cleaned = normalise.clean_datatypes(df_2_normalised, data_source_config.columns)
df_2_cleaned[df_2_cleaned["Child Unique ID"].isin({1007,1008,1009})]

Notice the removed erroneous date, and the intentional ID error there as well (actually - I messed up)? Hopefully the merge should get rid of the duplicates 🤞

## Join them up!

In [ ]:
merge.merge_dataframes([df_1_cleaned, df_2_cleaned], data_source_config.columns)


That's great! But... it has used the slightly older erroneous data where instead of the newer one. That's because we didn't use our sort key!

In [ ]:
df_1_cleaned['sort_key'] = list_2[0].sheet.sheet_detail.sort_key
df_2_cleaned['sort_key'] = list_2[1].sheet.sheet_detail.sort_key

merge.merge_dataframes([df_1_cleaned, df_2_cleaned], data_source_config.columns, sort_key='sort_key')